In [7]:
import pandas as pd 
import numpy as np

In [8]:
df = pd.read_csv("clean_data/final_flight_data.csv")

In [9]:
df.columns

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
       'Reporting_Airline', 'Origin', 'Dest', 'DepDelayMinutes',
       'ArrDelayMinutes', 'Cancelled', 'CarrierDelay', 'WeatherDelay',
       'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'orgin_lat',
       'orgin_long', 'des_lat', 'des_long'],
      dtype='object')

In [10]:
df_2017 = df[df['Year'] == 2017]

In [11]:
#to get 2017 total flight 
total_flight_2017 = df_2017['Reporting_Airline'].value_counts()
total_flight_2017.to_frame("total_count").reset_index().sort_values('index')

,index,total_count
1,AA,159210
7,AS,30276
5,B6,70809
3,DL,123165
6,EV,66125
9,F9,20984
11,HA,9104
8,NK,29998
2,OO,128688
4,UA,96347


In [12]:
#total late arrival 2017 (late)
total_late_arrival_2017 = df_2017[df_2017['ArrDelayMinutes'] > 15]

total_late_arrival_2017_count = total_late_arrival_2017['Reporting_Airline'].value_counts()
total_late_arrival_2017_count.to_frame("total_count").reset_index().sort_values('index')

#percentage of 2017 late arrival 
total_late_arrival_2017_percent = round((total_late_arrival_2017_count/total_flight_2017)*100,2)\
                    .sort_values(ascending = False)

on_time_arrival_2017_percent = 100 - total_late_arrival_2017_percent
on_time_arrival_2017_percent = on_time_arrival_2017_percent.reset_index().sort_values('index')

In [13]:
on_time_arrival_2017_percent = on_time_arrival_2017_percent.reset_index(drop = True)

In [14]:
#left on time arrive late
dep_ontime_arrive_late_2017 = df_2017[(df_2017['DepDelayMinutes'] < 15) & (df_2017['ArrDelayMinutes'] > 15)]

dep_ontime_arrive_late_2017_count = dep_ontime_arrive_late_2017['Reporting_Airline'].value_counts()
dep_ontime_arrive_late_2017_count.to_frame("total_count").reset_index().sort_values('index')

#Percentage of dep_ontime_arrive_late_2017
dep_ontime_arrive_late_2017_percent = round((dep_ontime_arrive_late_2017_count/total_flight_2017)*100,2)\
                    .sort_values(ascending = False)

dep_ontime_arrive_late_2017_percent

AS    32.53
HA    30.77
AA    26.93
VX    22.79
F9    20.91
UA    20.68
OO    20.68
DL    19.68
NK    18.02
EV    16.67
B6    15.23
WN    14.86
Name: Reporting_Airline, dtype: float64

In [15]:
# Calculate percentage depart ontime but arrived late
dep_ontime_arrive_late_2017_type_delay = dep_ontime_arrive_late_2017[['Reporting_Airline','CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay','LateAircraftDelay']]\
                        .groupby('Reporting_Airline').agg('sum').reset_index()
dep_ontime_arrive_late_2017_type_delay['total'] = dep_ontime_arrive_late_2017_type_delay['CarrierDelay'] + \
        dep_ontime_arrive_late_2017_type_delay['WeatherDelay'] + dep_ontime_arrive_late_2017_type_delay['NASDelay'] + \
        dep_ontime_arrive_late_2017_type_delay['SecurityDelay'] + dep_ontime_arrive_late_2017_type_delay['LateAircraftDelay']
dep_ontime_arrive_late_2017_type_delay_fin = dep_ontime_arrive_late_2017_type_delay[['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay','LateAircraftDelay']]\
                                            .div(dep_ontime_arrive_late_2017_type_delay.total*0.01, axis=0)
dep_ontime_arrive_late_2017_type_delay_fin['Reporting_Airline'] = dep_ontime_arrive_late_2017_type_delay['Reporting_Airline']
dep_ontime_arrive_late_2017_type_delay_fin

,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Reporting_Airline
0,6.519685,0.531527,89.739331,0.077904,3.131554,AA
1,5.308719,0.599391,89.511867,0.296876,4.283148,AS
2,9.249837,0.232074,88.748333,0.081835,1.687922,B6
3,5.935989,0.932381,90.827716,0.000000,2.303914,DL
4,6.454943,0.166984,91.002727,0.000000,2.375346,EV
5,7.133874,0.257027,90.957681,0.000000,1.651418,F9
6,77.544715,7.133229,5.137137,0.186282,9.998637,HA
7,4.267862,0.501248,93.473052,0.097728,1.660109,NK
8,3.715512,0.628082,89.160796,0.078989,6.416620,OO
9,5.308075,0.638877,92.718604,0.000000,1.334445,UA


In [16]:
final_airline_df_1 = pd.DataFrame(dep_ontime_arrive_late_2017_percent).reset_index()
final_airline_df_1.columns = ['Reporting_Airline', 'Per_Dep_Ontime_Arr_Late']
final_airline_df_1 = final_airline_df_1.sort_values('Reporting_Airline').reset_index(drop = True)

In [56]:
# final_airline_df['Arrival_Ontime'] = on_time_arrival_2017_percent['Reporting_Airline']

In [17]:
final_airline_df_2017 = final_airline_df_1.merge(dep_ontime_arrive_late_2017_type_delay_fin,  on = 'Reporting_Airline')

In [20]:
final_airline_df_2017['Year'] = 2017
final_airline_df_2017

,Reporting_Airline,Per_Dep_Ontime_Arr_Late,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Year
0,AA,26.93,6.519685,0.531527,89.739331,0.077904,3.131554,2017
1,AS,32.53,5.308719,0.599391,89.511867,0.296876,4.283148,2017
2,B6,15.23,9.249837,0.232074,88.748333,0.081835,1.687922,2017
3,DL,19.68,5.935989,0.932381,90.827716,0.000000,2.303914,2017
4,EV,16.67,6.454943,0.166984,91.002727,0.000000,2.375346,2017
5,F9,20.91,7.133874,0.257027,90.957681,0.000000,1.651418,2017
6,HA,30.77,77.544715,7.133229,5.137137,0.186282,9.998637,2017
7,NK,18.02,4.267862,0.501248,93.473052,0.097728,1.660109,2017
8,OO,20.68,3.715512,0.628082,89.160796,0.078989,6.416620,2017
9,UA,20.68,5.308075,0.638877,92.718604,0.000000,1.334445,2017


In [21]:
df_2018 = df[df['Year'] == 2018]

In [22]:
#to get 2018 total flight 
total_flight_2018 = df_2018['Reporting_Airline'].value_counts()
total_flight_2018.to_frame("total_count").reset_index().sort_values('index')

,index,total_count
9,9E,43660
1,AA,178272
12,AS,40272
5,B6,75123
3,DL,129486
11,EV,41470
13,F9,33398
15,G4,21102
16,HA,8563
6,MQ,57109


In [23]:
#left on time arrive late
dep_ontime_arrive_late_2018 = df_2018[(df_2018['DepDelayMinutes'] < 15) & (df_2018['ArrDelayMinutes'] > 15)]

dep_ontime_arrive_late_2018_count = dep_ontime_arrive_late_2018['Reporting_Airline'].value_counts()
dep_ontime_arrive_late_2018_count.to_frame("total_count").reset_index().sort_values('index')

#Percentage of dep_ontime_arrive_late_2017
dep_ontime_arrive_late_2018_percent = round((dep_ontime_arrive_late_2018_count/total_flight_2018)*100,2)\
                    .sort_values(ascending = False)

dep_ontime_arrive_late_2018_percent

AS    33.66
HA    33.55
VX    29.07
YX    26.30
MQ    24.64
UA    24.49
AA    23.99
DL    22.75
OO    22.74
NK    21.97
YV    21.14
EV    20.74
9E    18.98
OH    17.80
B6    17.33
F9    16.15
G4    15.66
WN    12.87
Name: Reporting_Airline, dtype: float64

In [24]:
# Calculate percentage depart ontime but arrived late
dep_ontime_arrive_late_2018_type_delay = dep_ontime_arrive_late_2018[['Reporting_Airline','CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay','LateAircraftDelay']]\
                        .groupby('Reporting_Airline').agg('sum').reset_index()
dep_ontime_arrive_late_2018_type_delay['total'] = dep_ontime_arrive_late_2018_type_delay['CarrierDelay'] + \
        dep_ontime_arrive_late_2018_type_delay['WeatherDelay'] + dep_ontime_arrive_late_2018_type_delay['NASDelay'] + \
        dep_ontime_arrive_late_2018_type_delay['SecurityDelay'] + dep_ontime_arrive_late_2018_type_delay['LateAircraftDelay']
dep_ontime_arrive_late_2018_type_delay_fin = dep_ontime_arrive_late_2018_type_delay[['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay','LateAircraftDelay']]\
                                            .div(dep_ontime_arrive_late_2018_type_delay.total*0.01, axis=0)
dep_ontime_arrive_late_2018_type_delay_fin['Reporting_Airline'] = dep_ontime_arrive_late_2018_type_delay['Reporting_Airline']
dep_ontime_arrive_late_2018_type_delay_fin

,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Reporting_Airline
0,3.537507,0.674153,91.924527,0.000000,3.863813,9E
1,6.870126,0.654964,89.192651,0.051739,3.230520,AA
2,4.416398,0.338320,91.250242,0.226388,3.768652,AS
3,9.758104,0.163711,87.853212,0.071272,2.153701,B6
4,5.819840,0.852086,90.787384,0.001510,2.539180,DL
5,5.267347,0.335502,92.026745,0.000000,2.370406,EV
6,7.987077,0.329503,89.344141,0.000000,2.339279,F9
7,8.875695,5.713856,77.717893,0.352150,7.340405,G4
8,81.218948,5.506494,5.272081,0.154801,7.847676,HA
9,4.655195,1.464913,89.372655,0.054537,4.452700,MQ


In [25]:
final_airline_df_2 = pd.DataFrame(dep_ontime_arrive_late_2018_percent).reset_index()
final_airline_df_2.columns = ['Reporting_Airline', 'Per_Dep_Ontime_Arr_Late']
final_airline_df_2 = final_airline_df_2.sort_values('Reporting_Airline').reset_index(drop = True)

In [26]:
final_airline_df_2018 = final_airline_df_2.merge(dep_ontime_arrive_late_2018_type_delay_fin,  on = 'Reporting_Airline')

In [28]:
final_airline_df_2018['Year'] = 2018
final_airline_df_2018

,Reporting_Airline,Per_Dep_Ontime_Arr_Late,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Year
0,9E,18.98,3.537507,0.674153,91.924527,0.000000,3.863813,2018
1,AA,23.99,6.870126,0.654964,89.192651,0.051739,3.230520,2018
2,AS,33.66,4.416398,0.338320,91.250242,0.226388,3.768652,2018
3,B6,17.33,9.758104,0.163711,87.853212,0.071272,2.153701,2018
4,DL,22.75,5.819840,0.852086,90.787384,0.001510,2.539180,2018
5,EV,20.74,5.267347,0.335502,92.026745,0.000000,2.370406,2018
6,F9,16.15,7.987077,0.329503,89.344141,0.000000,2.339279,2018
7,G4,15.66,8.875695,5.713856,77.717893,0.352150,7.340405,2018
8,HA,33.55,81.218948,5.506494,5.272081,0.154801,7.847676,2018
9,MQ,24.64,4.655195,1.464913,89.372655,0.054537,4.452700,2018


In [36]:

final_df = final_airline_df_2017.append(final_airline_df_2018)
final_df = final_df.reset_index(drop = True)
final_df.to_csv("clean_data/Delay_By_Airline.csv",index = False)


In [37]:
final_df

,Reporting_Airline,Per_Dep_Ontime_Arr_Late,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,Year
0,AA,26.93,6.519685,0.531527,89.739331,0.077904,3.131554,2017
1,AS,32.53,5.308719,0.599391,89.511867,0.296876,4.283148,2017
2,B6,15.23,9.249837,0.232074,88.748333,0.081835,1.687922,2017
3,DL,19.68,5.935989,0.932381,90.827716,0.000000,2.303914,2017
4,EV,16.67,6.454943,0.166984,91.002727,0.000000,2.375346,2017
5,F9,20.91,7.133874,0.257027,90.957681,0.000000,1.651418,2017
6,HA,30.77,77.544715,7.133229,5.137137,0.186282,9.998637,2017
7,NK,18.02,4.267862,0.501248,93.473052,0.097728,1.660109,2017
8,OO,20.68,3.715512,0.628082,89.160796,0.078989,6.416620,2017
9,UA,20.68,5.308075,0.638877,92.718604,0.000000,1.334445,2017
